In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder

In [2]:
# !kaggle competitions download -c titanic

In [3]:
train_data = pd.DataFrame(pd.read_csv('train.csv'))
test_data = pd.DataFrame(pd.read_csv('test.csv'))

In [4]:
test_data.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [5]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   PassengerId  891 non-null    int64  
 1   Survived     891 non-null    int64  
 2   Pclass       891 non-null    int64  
 3   Name         891 non-null    object 
 4   Sex          891 non-null    object 
 5   Age          714 non-null    float64
 6   SibSp        891 non-null    int64  
 7   Parch        891 non-null    int64  
 8   Ticket       891 non-null    object 
 9   Fare         891 non-null    float64
 10  Cabin        204 non-null    object 
 11  Embarked     889 non-null    object 
dtypes: float64(2), int64(5), object(5)
memory usage: 83.7+ KB


In [6]:
train_data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [7]:
def title_transform(x):
    if x == 'Mr':
        return x
    elif x in ['Mrs', 'Miss', 'Mme','Ms','Lady', 'Mlle', 'the Countess']:
        return 'Ms'
    elif x == 'Master':
        return x
    else:
        return 'Rare'

In [8]:
train_data['Title'] = train_data.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)
train_data['Title'] = train_data.Title.apply(title_transform)
train_data.drop('Name', axis=1, inplace=True)
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Ms
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Ms
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr


In [9]:
train_data['has_cabin'] = train_data.Cabin.notna().astype(int)
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,has_cabin
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Ms,1
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms,0
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Ms,1
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,0


In [10]:
train_data['family_size'] = train_data['SibSp']+ train_data['Parch']
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,has_cabin,family_size
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,1
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Ms,1,1
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms,0,0
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Ms,1,1
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,0,0


In [11]:
train_data['is_alone'] = train_data['family_size'].eq(0).astype(int)
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,has_cabin,family_size,is_alone
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,1,0
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Ms,1,1,0
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms,0,0,1
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Ms,1,1,0
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,0,0,1


In [12]:
missing = ['Sex', 'Embarked', 'Title']
train_data[missing] = train_data[missing].fillna('missed')
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,has_cabin,family_size,is_alone
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,1,0
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Ms,1,1,0
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms,0,0,1
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Ms,1,1,0
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,0,0,1


In [13]:
new_missing = ['Sex_f', 'Embarked_f', 'Title_f']
train_data[new_missing] = pd.DataFrame([[np.nan, np.nan, np.nan]], index=train_data.index)
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,has_cabin,family_size,is_alone,Sex_f,Embarked_f,Title_f
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,1,0,NaN,NaN,NaN
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Ms,1,1,0,NaN,NaN,NaN
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms,0,0,1,NaN,NaN,NaN
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Ms,1,1,0,NaN,NaN,NaN
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,0,0,1,NaN,NaN,NaN


In [14]:
ordinal_encoder = OrdinalEncoder()
train_data[new_missing] = ordinal_encoder.fit_transform(train_data[missing])
train_data.head()

,PassengerId,Survived,Pclass,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,Title,has_cabin,family_size,is_alone,Sex_f,Embarked_f,Title_f
0,1,0,3,male,22.0,1,0,A/5 21171,7.2500,NaN,S,Mr,0,1,0,1.0,2.0,1.0
1,2,1,1,female,38.0,1,0,PC 17599,71.2833,C85,C,Ms,1,1,0,0.0,0.0,2.0
2,3,1,3,female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Ms,0,0,1,0.0,2.0,2.0
3,4,1,1,female,35.0,1,0,113803,53.1000,C123,S,Ms,1,1,0,0.0,2.0,2.0
4,5,0,3,male,35.0,0,0,373450,8.0500,NaN,S,Mr,0,0,1,1.0,2.0,1.0


In [15]:
train_data.drop(missing, inplace=True, axis=1)
train_data.drop(['Cabin', 'Ticket'], inplace=True, axis=1)
train_data.head()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare,has_cabin,family_size,is_alone,Sex_f,Embarked_f,Title_f
0,1,0,3,22.0,1,0,7.2500,0,1,0,1.0,2.0,1.0
1,2,1,1,38.0,1,0,71.2833,1,1,0,0.0,0.0,2.0
2,3,1,3,26.0,0,0,7.9250,0,0,1,0.0,2.0,2.0
3,4,1,1,35.0,1,0,53.1000,1,1,0,0.0,2.0,2.0
4,5,0,3,35.0,0,0,8.0500,0,0,1,1.0,2.0,1.0


In [16]:
y = train_data.Survived
X = train_data.drop(['Survived', 'PassengerId'], axis=1)
X.isnull().sum()

Pclass           0
Age            177
SibSp            0
Parch            0
Fare             0
has_cabin        0
family_size      0
is_alone         0
Sex_f            0
Embarked_f       0
Title_f          0
dtype: int64

In [17]:
x_train, x_val, y_train, y_val = train_test_split(X, y, train_size=0.8,
                                                  test_size=0.2, random_state=1)

In [18]:
my_imputer = SimpleImputer()
imp_x_train = pd.DataFrame(my_imputer.fit_transform(x_train))
imp_x_val = pd.DataFrame(my_imputer.transform(x_val))
imp_x_train.columns = x_train.columns
imp_x_val.columns = x_val.columns

In [19]:
def mae_rf(i, j=None, k=None):
    model_rf = RandomForestClassifier(random_state=1, n_estimators=i, max_leaf_nodes=j, max_depth=k)
    model_rf.fit(imp_x_train, y_train)
    rf_pred = model_rf.predict(imp_x_val)
    return mean_absolute_error(rf_pred, y_val)

In [20]:
ls1 = [5, 10, 25, 50, 100, 200, 400, 500, 1000]
dictir_1 = {}
dictir = {}
for i in ls1:
    for j in ls1:
        for k in ls1:
            tmp = mae_rf(i, j, k)
            if i not in dictir_1:
                dictir_1[i] = tmp
                dictir[i] = [j, k]
                continue
            if tmp < dictir_1[i]:
                dictir_1[i] = tmp
                dictir[i] = [j, k]
                
print(min(dictir_1, key=dictir_1.get))
dictir_1[min(dictir_1, key=dictir_1.get)]

10


0.18435754189944134

In [21]:
l = min(dictir_1, key=dictir_1.get)
print(l, dictir[l][1], dictir[l][0])

10 5 10


In [32]:
l2, l3 = dictir[l][0], dictir[l][1]
mae_rf(l, l2, l3)

0.18435754189944134

In [33]:
my_mod = RandomForestClassifier(random_state=l, n_estimators=l2, max_leaf_nodes=l3, max_depth=15)
my_mod.fit(imp_x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=15, max_features='auto',
                       max_leaf_nodes=5, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=10,
                       n_jobs=None, oob_score=False, random_state=10, verbose=0,
                       warm_start=False)

In [34]:
test_data['Title'] = test_data.Name.str.extract(r',\s*([^\.]*)\s*\.', expand=False)
test_data['Title'] = test_data.Title.apply(title_transform)
test_data.drop('Name', axis=1, inplace=True)

test_data['has_cabin'] = test_data.Cabin.notna().astype(int)

test_data['family_size'] = test_data['SibSp']+ test_data['Parch']

test_data['is_alone'] = test_data['family_size'].eq(0).astype(int)

test_data[missing] = test_data[missing].fillna('missed')

test_data[new_missing] = pd.DataFrame([[np.nan, np.nan, np.nan]], index=test_data.index)

test_data[new_missing] = ordinal_encoder.fit_transform(test_data[missing])
test = test_data.copy()
test.drop(missing, inplace=True, axis=1)
test.drop(['Cabin', 'Ticket'], inplace=True, axis=1)

AttributeError: 'DataFrame' object has no attribute 'Name'

In [24]:
test.drop(['PassengerId'], inplace=True, axis=1)

In [35]:
test.isnull().sum()

Pclass          0
Age            86
SibSp           0
Parch           0
Fare            1
has_cabin       0
family_size     0
is_alone        0
Sex_f           0
Embarked_f      0
Title_f         0
dtype: int64

In [36]:
test_imp = pd.DataFrame(my_imputer.transform(test))
test_imp.isnull().sum()

0     0
1     0
2     0
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
dtype: int64

In [37]:
fin_pred = my_mod.predict(test_imp)

In [38]:
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': fin_pred})
output.to_csv('submission.csv', index=False)

In [39]:
!kaggle competitions submit -c titanic -f submission.csv -m "sklearn try"

100%|████████████████████████████████████████| 2.77k/2.77k [00:03<00:00, 750B/s]
Successfully submitted to Titanic: Machine Learning from Disaster